In [1]:
from pathlib import Path
from kedro.framework.session import KedroSession
from kedro.framework.session.session import _activate_session
from kedro.framework.startup import _add_src_to_path

current_dir = Path.cwd()  # this points to 'notebooks/' folder
project_path = current_dir.parent  # point back to the root of the project
_add_src_to_path(project_path/"src", project_path)

session = KedroSession.create("csmr_kedro", project_path)
_activate_session(session)
context = session.load_context()
catalog = context.catalog


2022-01-27 10:17:34,910 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.


RuntimeError: Cannot activate the session as another active session already exists.

In [1]:
catalog

In [2]:
train_features  = catalog.load("train_features")

2022-01-27 18:57:28,833 - kedro.io.data_catalog - INFO - Loading data from `train_features` (CSVDataSet)...


In [3]:
train_features.dtypes

feature_1      float64
feature_2      float64
feature_3      float64
feature_4      float64
feature_5      float64
                ...   
feature_767    float64
feature_768    float64
text            object
company         object
target         float64
Length: 771, dtype: object

In [6]:
import torch
torch.cuda.is_available()

True

In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline

numeric_features = list(filter(lambda x: x.startswith("feature_"), train_features.columns))
categorical_features = ["company"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", 'passthrough', numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

pipe = Pipeline(steps=[
    ('data_transformer', preprocessor),
  ('estimator', RidgeClassifier())
])

In [26]:
X = train_features.drop(columns=["target", "text"])
categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

In [33]:
pipe.fit(X, train_features.target)

Pipeline(steps=[('data_transformer',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['feature_1', 'feature_2',
                                                   'feature_3', 'feature_4',
                                                   'feature_5', 'feature_6',
                                                   'feature_7', 'feature_8',
                                                   'feature_9', 'feature_10',
                                                   'feature_11', 'feature_12',
                                                   'feature_13', 'feature_14',
                                                   'feature_15', 'feature_16',
                                                   'feature_17', 'feature_18',
                                                   'feature_19', 'feature_20',
                                                   'feature_21', 'feature_22',
                               

In [30]:
out = preprocessor.fit_transform(train_features.drop(columns=["target", "text"]))
out.shape

(640, 775)

In [16]:
out

<640x7 sparse matrix of type '<class 'numpy.float64'>'
	with 640 stored elements in Compressed Sparse Row format>